In [35]:
import pandas as pd
import dash
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

df_dejavnosti = pd.read_csv("data/podatki_po_dejavnostih.csv", delimiter=",", encoding="windows-1250")
df_odjemalci = pd.read_csv("data/podatki_vrsta_odjemalca.csv", delimiter=",", encoding="windows-1250")

In [36]:
df_odjemalci_backup = df_odjemalci.copy()

#Tukaj vzamemo samo podatke za VSE SKUPAJ porabo (brez gospodinjstva in poslovnih objektov)
df = df_odjemalci_backup[::+3]
#Tukaj izbrišem podatke za SLOVENIJO in Neznano vrstice
df = df[1:-1]

#Priprava podatkov za uporabo
df = df.drop("VRSTA ODJEMALCA", axis=1)
obcine = {df["OBČINE"][i]:i for i in df.index}

vse_2020 = dict(df[df.columns[1:13]].sum(axis=1))
vse_2021 = dict(df[df.columns[13:25]].sum(axis=1))
vse_2022 = dict(df[df.columns[25:34]].sum(axis=1))
dict_porab = dict([(k,[vse_2020[k], vse_2021[k], vse_2022[k]]) for k in vse_2020])

#V koncem dictu so shranjene OBČINE (ključ) z vrednostmi, ki so LISTI, 
# ki prikazujejo porabo za vsako občino za vsako leto posebaj
koncni_dict = dict()
for k,v in obcine.items():
    koncni_dict[k] = dict_porab[v]

In [89]:
def izrisi_odjemalci_Slovenija(odjemalci, leto):
    df_odjemalci = odjemalci.copy()
    df=df_odjemalci[::+3]
    df = df[1:-1]
    df = df.drop("VRSTA ODJEMALCA", axis=1)
    obcine = {df["OBČINE"][i]:i for i in df.index}

    dict_porab = dict()
    if leto == 2020:
        vse_2020 = dict(df[df.columns[1:13]].sum(axis=1))
        dict_porab = dict([(k,[vse_2020[k]]) for k in vse_2020])
    elif leto == 2021:
        vse_2021 = dict(df[df.columns[13:25]].sum(axis=1))
        dict_porab = dict([(k,[vse_2021[k]]) for k in vse_2021])

    koncni_dict = dict()
    for k,v in obcine.items():
        koncni_dict[k] = dict_porab[v]
    
    return pd.DataFrame.from_dict(koncni_dict)

In [92]:
def izrisi_odjemalci_gospodinjstva(odjemalci, leto):
    df_odjemalci = odjemalci.copy()
    df=df_odjemalci[1::+3]
    df = df[1:-1]
    df = df.drop("VRSTA ODJEMALCA", axis=1)
    obcine = {df["OBČINE"][i]:i for i in df.index}

    dict_porab = dict()
    if leto == 2020:
        vse_2020 = dict(df[df.columns[1:13]].sum(axis=1))
        dict_porab = dict([(k,[vse_2020[k]]) for k in vse_2020])
    elif leto == 2021:
        vse_2021 = dict(df[df.columns[13:25]].sum(axis=1))
        dict_porab = dict([(k,[vse_2021[k]]) for k in vse_2021])

    koncni_dict = dict()
    for k,v in obcine.items():
        koncni_dict[k] = dict_porab[v]

    return pd.DataFrame.from_dict(koncni_dict)

In [94]:
def izrisi_odjemalci_poslovni_objekti(odjemalci, leto):
    df_odjemalci = odjemalci.copy()
    df=df_odjemalci[2::+3]
    df = df[1:-1]
    df = df.drop("VRSTA ODJEMALCA", axis=1)
    obcine = {df["OBČINE"][i]:i for i in df.index}

    dict_porab = dict()
    if leto == 2020:
        vse_2020 = dict(df[df.columns[1:13]].sum(axis=1))
        dict_porab = dict([(k,[vse_2020[k]]) for k in vse_2020])
    elif leto == 2021:
        vse_2021 = dict(df[df.columns[13:25]].sum(axis=1))
        dict_porab = dict([(k,[vse_2021[k]]) for k in vse_2021])

    koncni_dict = dict()
    for k,v in obcine.items():
        koncni_dict[k] = dict_porab[v]

    return pd.DataFrame.from_dict(koncni_dict)

In [99]:
#Izris podatkov PRIMERI (2020 in 2021)
izrisi_odjemalci_Slovenija(df_odjemalci, 2020)
#izrisi_odjemalci_gospodinjstva(df_odjemalci, 2020)
#izrisi_odjemalci_poslovni_objekti(df_odjemalci, 2020)

,Ajdovščina,Ankaran/Ancarano,Apače,Beltinci,Benedikt,Bistrica ob Sotli,Bled,Bloke,Bohinj,Borovnica,...,Vuzenica,Zagorje ob Savi,Zavrč,Zreče,Žalec,Železniki,Žetale,Žiri,Žirovnica,Žužemberk
0,93181950,19112313,10051617,24589761,6607746,3319459,38934450,6787877,28971836,19832338,...,31837039,60507390,2988233,86808888,131915162,38035966,2495314,24799402,18732777,17195867
